In [1]:
import xarray as xr
import numpy as np
import glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import netCDF4
import pandas as pd
import xesmf as xe
from scipy import signal

In [3]:
var = "thetao"
arr = []
for ens_index in range(1,81):
      for ly in range(1,11):
        ly_string = "ly"+str(ly).zfill(2)
        filename = "/work/uo1075/u241321/data/u241321/data_cdo/thetao_80/"+var+"_Omon_MPI-ESM-LR_dkfen41960_2018_r"+str(ens_index)+"i2p1_ym_"+str(ly_string)+"_g1d0_NA.nc"
        arr.append(xr.open_dataset(filename,decode_times=False).assign_coords(ensemble=ens_index,ly=ly))

In [4]:
data = xr.concat(arr, dim="ensemble")

<xarray.Dataset>
Dimensions:    (time: 68, lon: 150, lat: 90, depth: 40, ensemble: 800, bnds: 2)
Coordinates:
  * time       (time) float64 7.5 19.5 31.5 43.5 ... 775.5 787.5 799.5 811.5
  * lon        (lon) float64 -79.5 -78.5 -77.5 -76.5 ... 66.5 67.5 68.5 69.5
  * lat        (lat) float64 0.5 1.5 2.5 3.5 4.5 ... 85.5 86.5 87.5 88.5 89.5
  * depth      (depth) float64 6.0 17.0 27.0 37.0 ... 4.67e+03 5.17e+03 5.72e+03
  * ensemble   (ensemble) int64 1 1 1 1 1 1 1 1 1 ... 80 80 80 80 80 80 80 80 80
    ly         (ensemble) int64 1 2 3 4 5 6 7 8 9 10 1 ... 1 2 3 4 5 6 7 8 9 10
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (ensemble, time, bnds) float64 2.0 13.0 14.0 ... 806.0 817.0
    thetao     (ensemble, time, depth, lat, lon) float32 nan nan nan ... nan nan
Attributes:
    CDI:          Climate Data Interface version 2.2.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.4
    MPIOM:        $Revision: cosmos/tags/mpiesm-1.2.01p5 9876 mpiom/tags/mpio...
    history:      Fri Jan 05 16:30:48 2024: cdo -sellonlatbox,-80,70,0,90 the...
    CDO:          Climate Data Operators version 2.2.2 (https://mpimet.mpg.de...

In [5]:
ds = data ['thetao'][:,9:59,:,:,:]
ds.to_netcdf("/work/uo1075/u241321/data/thetao_ly_1969-2019_hindcast_80_step1.nc")

In [6]:
# A reshaped DataArray. In the example in the function comment it would correspond to an array like

#In[1] Ar.dims
#Out[1]: ('year', 'month', 'lat', 'lon')

def xr_reshape(A, dim, newdims, coords):
    """ Reshape DataArray A to convert its dimension dim into sub-dimensions given by
    newdims and the corresponding coords.
    Example: Ar = xr_reshape(A, 'time', ['year', 'month'], [(2017, 2018), np.arange(12)]) """


    # Create a pandas MultiIndex from these labels
    ind = pd.MultiIndex.from_product(coords, names=newdims)

    # Replace the time index in the DataArray by this new index,
    A1 = A.copy()

    A1.coords[dim] = ind

    # Convert multiindex to individual dims using DataArray.unstack().
    # This changes dimension order! The new dimensions are at the end.
    A1 = A1.unstack(dim)

    # Permute to restore dimensions
    i = A.dims.index(dim)
    dims = list(A1.dims)

    for d in newdims[::-1]:
        dims.insert(i, d)

    for d in newdims:
        _ = dims.pop(-1)


    return A1.transpose(*dims)




In [7]:
ds_r = xr_reshape(ds, 'ensemble', ['member', 'lead_year'], [np.arange(1,81),np.arange(1,11),])

MemoryError: Unable to allocate 80.5 GiB for an array with shape (50, 40, 90, 150, 80, 10) and data type float32

In [9]:
annual = ds_r
annual_pre = np.zeros((ds_r['member'].size,ds_r['lead_year'].size,50,ds_r['lat'].size,ds_r['lon'].size))
annual_pre = annual[:,:,9:59,:,:]

annual_prediction = annual_pre.transpose("time", "member", "lead_year", "lat", "lon") # 
dat = np.mean(annual_prediction,axis=0)
dif = annual_prediction-dat
dif

<xarray.DataArray 'psl' (time: 50, member: 80, lead_year: 10, lat: 180, lon: 360)>
array([[[[[-8.45078125e+01, -8.45078125e+01, -8.41796875e+01, ...,
           -8.50703125e+01, -8.48281250e+01, -8.48281250e+01],
          [-8.49453125e+01, -8.47578125e+01, -8.45703125e+01, ...,
           -8.53437500e+01, -8.52187500e+01, -8.50937500e+01],
          [-8.91328125e+01, -8.89218750e+01, -8.87031250e+01, ...,
           -8.97812500e+01, -8.95781250e+01, -8.93515625e+01],
          ...,
          [-6.18906250e+01, -6.22656250e+01, -6.26562500e+01, ...,
           -6.07265625e+01, -6.11015625e+01, -6.15000000e+01],
          [-4.86328125e+01, -4.88671875e+01, -4.91093750e+01, ...,
           -4.78828125e+01, -4.81406250e+01, -4.83984375e+01],
          [-4.77890625e+01, -4.77890625e+01, -4.82031250e+01, ...,
           -4.68906250e+01, -4.73359375e+01, -4.73359375e+01]],

         [[ 2.34460938e+02,  2.34460938e+02,  2.35359375e+02, ...,
            2.32718750e+02,  2.33578125e+02,  2.33578125e+02],
          [ 2.34515625e+02,  2.35070312e+02,  2.35625000e+02, ...,
            2.32882812e+02,  2.33421875e+02,  2.33953125e+02],
          [ 2.38289062e+02,  2.39929688e+02,  2.41585938e+02, ...,
            2.33390625e+02,  2.35031250e+02,  2.36656250e+02],
...
            1.81867188e+02,  1.81531250e+02,  1.81187500e+02],
          [ 2.05406250e+02,  2.05289062e+02,  2.05203125e+02, ...,
            2.05726562e+02,  2.05617188e+02,  2.05492188e+02],
          [ 2.07125000e+02,  2.07132812e+02,  2.06984375e+02, ...,
            2.07507812e+02,  2.07312500e+02,  2.07312500e+02]],

         [[ 1.02601562e+02,  1.02609375e+02,  1.01625000e+02, ...,
            1.04414062e+02,  1.03539062e+02,  1.03539062e+02],
          [ 1.03429688e+02,  1.02875000e+02,  1.02359375e+02, ...,
            1.04781250e+02,  1.04328125e+02,  1.03882812e+02],
          [ 1.10679688e+02,  1.10445312e+02,  1.10226562e+02, ...,
            1.11179688e+02,  1.11015625e+02,  1.10867188e+02],
          ...,
          [ 4.34382812e+02,  4.33976562e+02,  4.33539062e+02, ...,
            4.35546875e+02,  4.35171875e+02,  4.34773438e+02],
          [ 4.33664062e+02,  4.33429688e+02,  4.33164062e+02, ...,
            4.34320312e+02,  4.34109375e+02,  4.33882812e+02],
          [ 4.33468750e+02,  4.33468750e+02,  4.33039062e+02, ...,
            4.34289062e+02,  4.33898438e+02,  4.33898438e+02]]]]],
      dtype=float32)
Coordinates:
  * member     (member) int64 1 2 3 4 5 6 7 8 9 ... 72 73 74 75 76 77 78 79 80
  * lead_year  (lead_year) int64 1 2 3 4 5 6 7 8 9 10
  * time       (time) float64 115.5 127.5 139.5 151.5 ... 679.5 691.5 703.5
  * lon        (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    ly         (member, lead_year) int64 1 2 3 4 5 6 7 8 9 ... 3 4 5 6 7 8 9 10

In [10]:
dif.to_netcdf("/work/uo1075/u241321/data/thetao_ly_1969-2019_hindcast80.nc")

In [13]:
dropped = annual_prediction.stack(feature=("member","lead_year","lat","lon")).dropna(dim="feature")
detrend = signal.detrend(dropped ,axis=0)
feature = dropped .coords["feature"]
time = dropped .coords["time"]
detrend = xr.DataArray(detrend, dims = ["time","feature"], coords = {"time":time,"feature":feature}).unstack()
detrend.to_netcdf("/work/uo1075/u241321/data/thetao_ly_1970-2019_hindcast80_dt.nc")